<a href="https://colab.research.google.com/github/bhutamanav11/YardStick/blob/main/yardstick_internship_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade openai


In [ ]:
!pip install --quiet --upgrade gitpython

In [ ]:
from openai import OpenAI

from google.colab import userdata
userdata.get('GROQ_API_KEY')
# Create the client (must use Groq base_url)
client = OpenAI(api_key=os.environ["GROQ_API_KEY"],
                base_url="https://api.groq.com/openai/v1")  # Groq OpenAI-compatible base URL. :contentReference[oaicite:4]{index=4}
print("Client created.")


Client created.


In [ ]:
# List available models so you can pick a supported one.
try:
    resp = client.models.list()
    models = [m.id for m in resp.data]
    print("Available models (sample):", models[:20])
except Exception as e:
    print("Error listing models:", e)
    raise


Available models (sample): ['whisper-large-v3-turbo', 'deepseek-r1-distill-llama-70b', 'moonshotai/kimi-k2-instruct-0905', 'llama-3.3-70b-versatile', 'groq/compound', 'meta-llama/llama-prompt-guard-2-22m', 'allam-2-7b', 'meta-llama/llama-prompt-guard-2-86m', 'moonshotai/kimi-k2-instruct', 'openai/gpt-oss-120b', 'playai-tts-arabic', 'llama-3.1-8b-instant', 'groq/compound-mini', 'meta-llama/llama-4-scout-17b-16e-instruct', 'openai/gpt-oss-20b', 'meta-llama/llama-4-maverick-17b-128e-instruct', 'gemma2-9b-it', 'whisper-large-v3', 'qwen/qwen3-32b', 'meta-llama/llama-guard-4-12b']


In [ ]:
# Example conversation list (store/manage chats as a list of dicts)
conversations = [
    {"role": "user", "content": "Hi, I need help finishing my internship assignment."},
    {"role": "assistant", "content": "Sure — what part are you stuck on?"},
    {"role": "user", "content": "I need to summarize chats and classify intent and extract structured details."},
    {"role": "user", "content": "Deadline is next Friday and my email is student@example.com."}
]

# Persist to local file (optional)
import json
with open("conversations.json","w") as f:
    json.dump(conversations, f, indent=2)
print("Saved conversations.json")


Saved conversations.json


In [ ]:
MODEL_ID = "llama-3.3-70b-versatile"

# Build messages: system instruction + conversation history
system_msg = {"role":"system","content":"You are a concise summarizer. Produce a short summary (3 bullets max)."}
msgs = [system_msg] + conversations

try:
    res = client.chat.completions.create(
        model=MODEL_ID,
        messages=msgs,
        temperature=0.0,
        max_tokens=256
    )
    summary = res.choices[0].message.content.strip()
    print("=== SUMMARY ===\n", summary)
except Exception as e:
    print("Summarization error:", e)
    raise


=== SUMMARY ===
 Here's a summary to help you get started:
* You need help with an internship assignment involving chat summaries.
* The tasks include classifying intent and extracting structured details from chats.
* Your deadline is next Friday, and I can follow up with you at student@example.com if needed.


In [ ]:
import re, json

# 1) Build the user-only prompt
user_msgs = [m["content"] for m in conversations if m["role"] == "user"]
prompt_text = "\n".join(f"{i+1}. {t}" for i,t in enumerate(user_msgs))

system_instructions = """
You are a strict JSON-only extractor. For each user message (provided), return a JSON object:
{
  "items": [
    {
      "index": <1-based index>,
      "text": "<original message>",
      "intent": "<short label: greeting|request|info|deadline|other>",
      "sentiment": "<positive|neutral|negative>",
      "entities": {
         "emails": ["..."],
         "dates": ["..."],
         "phones": ["..."],
         "deadlines": ["..."]
      }
    }, ...
  ]
}
Return ONLY the JSON. No extra text.
"""

msgs2 = [{"role":"system","content":system_instructions},
         {"role":"user","content":prompt_text}]

try:
    resp2 = client.chat.completions.create(
        model=MODEL_ID,
        messages=msgs2,
        temperature=0.0,
        max_tokens=512
    )
    raw = resp2.choices[0].message.content.strip()
    # extract JSON block
    m = re.search(r'(\{.*\}|\[.*\])', raw, re.S)
    if not m:
        raise ValueError("No JSON found in model response:\n" + raw)
    json_text = m.group(1)
    data = json.loads(json_text)
    print("=== CLASSIFICATION/EXTRACTION ===")
    print(json.dumps(data, indent=2))
except Exception as e:
    print("Classification/extraction error:", e)
    print("Raw model output:\n", raw if 'raw' in locals() else "no raw")
    raise


=== CLASSIFICATION/EXTRACTION ===
{
  "items": [
    {
      "index": 1,
      "text": "Hi, I need help finishing my internship assignment.",
      "intent": "request",
      "sentiment": "neutral",
      "entities": {
        "emails": [],
        "dates": [],
        "phones": [],
        "deadlines": []
      }
    },
    {
      "index": 2,
      "text": "I need to summarize chats and classify intent and extract structured details.",
      "intent": "info",
      "sentiment": "neutral",
      "entities": {
        "emails": [],
        "dates": [],
        "phones": [],
        "deadlines": []
      }
    },
    {
      "index": 3,
      "text": "Deadline is next Friday and my email is student@example.com.",
      "intent": "deadline",
      "sentiment": "neutral",
      "entities": {
        "emails": [
          "student@example.com"
        ],
        "dates": [
          "next Friday"
        ],
        "phones": [],
        "deadlines": [
          "next Friday"
        ]
    

In [ ]:
with open("summary.txt","w") as f:
    f.write(summary+"\n")

with open("classification.json","w") as f:
    json.dump(data, f, indent=2)

print("Saved summary.txt and classification.json")


Saved summary.txt and classification.json
